In [1]:
import sys
sys.path.append("../code")
from dataset import build_dataset, build_dataloader
from config import DataArguments, TrainerArguments, ModelArguments

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\cuda\__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
data_args = DataArguments
training_args = TrainerArguments
model_args = ModelArguments

data_args.data_dir = "../data/"
data_args.seed = training_args.seed
data_args.max_seq_len = model_args.max_seq_len

dataset, model_args.vocab_size, model_args.num_labels = build_dataset(data_args, "train")
dataloader = build_dataloader(dataset, data_args)

[11/20/2021 21:25:25] INFO - dataset: Initialize Train Dataset.
[11/20/2021 21:25:25] INFO - dataset: Remove abstract.
[11/20/2021 21:25:25] INFO - dataset: Remove task_id
[11/20/2021 21:25:25] INFO - dataset: Use 0.05% of the total dataset.
[11/20/2021 21:25:26] INFO - dataset: Total 2363 of papers will be used.
[11/20/2021 21:25:26] INFO - dataset: Train dataset was successfully initialized.
[11/20/2021 21:25:29] INFO - preprocess: Successfully loaded Spacy Tokenizer, en_core_web_trf
[11/20/2021 21:25:29] INFO - dataset: Successfully loaded mapper file ..\assets\area2idx.json
[11/20/2021 21:25:31] WARNING - datasets.fingerprint: Parameter 'function'=<function build_dataset.<locals>.batch_encode at 0x000001EAA610D558> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this tr

In [3]:
from load_model import load_model

# model_args.vocab_size = 83931
# model_args.model_name_or_path = 'bert'
# model_args.num_labels = 16
model = load_model(model_args)

In [4]:
batch = next(iter(dataloader))
logits, predicted_class = model(batch)

KeyError: 'label'

In [12]:
batch

{'input_ids': tensor([[    0, 35166,  3480,  ...,     0,     0,     0],
         [    0,   100,    12,  ...,     0,     0,     0],
         [    0, 24514, 21464,  ...,     0,     0,     0],
         ...,
         [    0, 43780,   154,  ...,     0,     0,     0],
         [    0, 14563,  7367,  ...,     0,     0,     0],
         [    0, 35166,  2239,  ...,     0,     0,     0]]),
 'label': tensor([ 7, 10,  7, 14, 13, 12, 14, 14,  7,  3, 10,  8, 10, 12, 14,  7,  6, 10,
          6,  7, 10,  3, 15, 15, 10, 10,  8,  7, 12, 15, 10, 14,  4,  7, 15,  3,
          3, 15, 10,  7, 14, 13, 15, 10, 12,  3,  7, 12, 13,  7,  5, 11, 12, 15,
          7,  7, 12,  4,  7,  7,  8,  3,  7,  5, 12, 15, 13,  3,  7,  4, 15, 12,
         12, 12, 14,  5, 11, 10,  4,  7, 13,  9,  4,  7,  7, 12, 11,  7, 10, 14,
         15,  3, 10, 15,  7,  4,  4,  8, 14,  7, 12,  5, 13,  7,  3,  7,  2, 14,
          4,  7,  7, 11, 15, 15,  6, 15, 14,  3,  7, 15, 15,  3, 10,  7, 12,  3,
          3,  6])}

In [16]:
out = model.model(**batch)
loss = out["loss"]
logits = out["logits"]

In [17]:
loss, logits

(tensor(2.8167, grad_fn=<NllLossBackward0>),
 tensor([[ 0.3428,  0.2857, -0.2186,  ..., -0.3939,  0.0539, -0.1727],
         [ 0.1125,  0.5036, -0.0297,  ..., -0.1842,  0.2904,  0.1124],
         [ 0.2734,  0.4260,  0.0040,  ..., -0.2952,  0.1796,  0.1000],
         ...,
         [ 0.0574,  0.4719, -0.1994,  ..., -0.0546,  0.2691, -0.0266],
         [ 0.1133,  0.2223, -0.0434,  ..., -0.1677,  0.2933,  0.1312],
         [ 0.3418,  0.6447, -0.1163,  ...,  0.1065,  0.1086, -0.0074]],
        grad_fn=<AddmmBackward0>))

In [18]:
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()

In [19]:
loss_fn(logits, batch["labels"])

tensor(2.8167, grad_fn=<NllLossBackward0>)